In [1]:
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.ml.recommendation.ALSModel
import org.apache.spark.ml.evaluation.RegressionEvaluator

In [2]:
val rawData = sc.textFile("/home/jovyan/data/u.data")
rawData.first()

rawData = /home/jovyan/data/u.data MapPartitionsRDD[1] at textFile at <console>:30


196	242	3	881250949

In [3]:
case class Rating(userId: Int, movieId: Int, rating: Float, timeStamp: Long)
def parseString(str: String): Rating = {
    val fields = str.split("\t")
    assert(fields.size==4)
    Rating(fields(0).toInt, fields(1).toInt, fields(2).toFloat, fields(3).toLong)
}

defined class Rating


parseString: (str: String)Rating


In [4]:
val ratings = rawData.map(parseString).toDF()
ratings.show(3)

+------+-------+------+---------+
|userId|movieId|rating|timeStamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
+------+-------+------+---------+
only showing top 3 rows



ratings = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [5]:
val Array(train, test) = ratings.randomSplit(Array(0.8, 0.2))
println(train.count, rawData.count)

(80022,100000)


train = [userId: int, movieId: int ... 2 more fields]
test = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [6]:
train.show(3)

+------+-------+------+---------+
|userId|movieId|rating|timeStamp|
+------+-------+------+---------+
|     1|      1|   5.0|874965758|
|     1|      2|   3.0|876893171|
|     1|      6|   5.0|887431973|
+------+-------+------+---------+
only showing top 3 rows



In [7]:
val als = new ALS().setMaxIter(5).setRegParam(0.01).setUserCol("userId").setItemCol("movieId").setRatingCol("rating")

als = als_968eb4569a60


als_968eb4569a60

In [8]:
val model = als.fit(train)

model = als_968eb4569a60


als_968eb4569a60

In [9]:
model.setColdStartStrategy("drop")
val predictions = model.transform(test)
predictions.show(3)

+------+-------+------+---------+----------+
|userId|movieId|rating|timeStamp|prediction|
+------+-------+------+---------+----------+
|   251|    148|   2.0|886272547|  2.979134|
|   602|    148|   4.0|888638517| 4.6452227|
|   663|    148|   4.0|889492989| 3.0142186|
+------+-------+------+---------+----------+
only showing top 3 rows



predictions = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [10]:
val eval = new RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
val rmse = eval.evaluate(predictions)

eval = regEval_2786e87f6c33
rmse = 1.054296159839924


1.054296159839924

In [11]:
val userRecs = model.recommendForAllUsers(10)
userRecs.filter(userRecs("userId").equalTo(471)).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[862, 14.712774]...|
+------+--------------------+



userRecs = [userId: int, recommendations: array<struct<movieId:int,rating:float>>]


[userId: int, recommendations: array<struct<movieId:int,rating:float>>]

In [12]:
val path = "/tmp/myrecmodel"
model.save(path)

Name: java.io.IOException
Message: Path /tmp/myrecmodel already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
StackTrace:   at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:503)
  at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:102)
  at org.apache.spark.ml.util.MLWritable$class.save(ReadWrite.scala:162)
  at org.apache.spark.ml.recommendation.ALSModel.save(ALS.scala:267)

In [13]:
val samemodel = ALSModel.load(path)

Name: Unknown Error
Message: lastException: Throwable = null
<console>:28: error: not found: value path
       val samemodel = ALSModel.load(path)
                                     ^

StackTrace: 